In [1]:
import Pkg; Pkg.add("ITensors")
import Pkg; Pkg.add("ITensorMPS")
import Pkg; Pkg.add("BlockArrays")
import Pkg; Pkg.add("LinearAlgebra")

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`


In [2]:
using ITensors, ITensorMPS

In [3]:
let
    A=1
  N = 14
  sites = siteinds("S=1/2",N)

  os = OpSum()
  for j=1:N-1
    os += 0.5,"S+",j,"S-",j+1
    os += 0.5,"S-",j,"S+",j+1
    os += "Sz",j,"Sz",j+1
  end
  H = MPO(os,sites)

  nsweeps = 15 # number of sweeps is 5
  maxdim = [10, 100, 1000, 10000, 25580] # gradually increase states kept
  cutoff = [1E-10] # desired truncation error

  psi0 = random_mps(sites;linkdims=10)

  energy,psi1 = dmrg(H,psi0;nsweeps,maxdim,cutoff)
  
a1=0
orthogonalize!(psi1,A)
ket1 = psi1[A]
bra1= dag(ket1)
rho = prime(ket1,"Site")*bra1 
da, ua = eigen(rho)
for j=1:length(diag(real(da)))
   a1+=diag(real(da))[j]
    end
    @show a1;
    return
end

After sweep 1 energy=-6.025987271627872  maxlinkdim=10 maxerr=1.79E-03 time=20.461
After sweep 2 energy=-6.026724647226379  maxlinkdim=29 maxerr=9.79E-11 time=0.070
After sweep 3 energy=-6.026724660078803  maxlinkdim=31 maxerr=8.97E-11 time=0.085
After sweep 4 energy=-6.026724660089144  maxlinkdim=31 maxerr=9.05E-11 time=0.138
After sweep 5 energy=-6.02672466008799  maxlinkdim=31 maxerr=9.05E-11 time=0.110
After sweep 6 energy=-6.026724660087297  maxlinkdim=31 maxerr=9.05E-11 time=0.089
After sweep 7 energy=-6.026724660087185  maxlinkdim=31 maxerr=9.05E-11 time=0.080
After sweep 8 energy=-6.026724660087313  maxlinkdim=31 maxerr=9.05E-11 time=0.109
After sweep 9 energy=-6.026724660087373  maxlinkdim=31 maxerr=9.05E-11 time=0.102
After sweep 10 energy=-6.02672466008741  maxlinkdim=31 maxerr=9.05E-11 time=0.092
After sweep 11 energy=-6.02672466008743  maxlinkdim=31 maxerr=9.05E-11 time=0.087
After sweep 12 energy=-6.026724660087414  maxlinkdim=31 maxerr=9.05E-11 time=0.077
After sweep 13 

In [4]:
```
To perform a partial trace I just need to specify the indices I would likde to trace out.
I should also keep in min that the indices form a pair as '(inds(rho)[i], inds(rho)[i+D])', where D represents the last site of interest.
Keep in mind that I am exclusively interested in 4-partite entanglement.
```

`To perform a partial trace I just need to specify the indices I would likde to trace out. I should also keep in min that the indices form a pair as '(inds(rho)[i], inds(rho)[i+D]),' where D represents the last site of interest. Keep in mind that I am exclusively interested in 4-partite entanglement.`

In [5]:
let
    A=1
    C=6
  N = 16
  sites = siteinds("S=1/2",N)

  os = OpSum()
  for j=1:N-1
    os += 0.5,"S+",j,"S-",j+1
    os += 0.5,"S-",j,"S+",j+1
    os += "Sz",j,"Sz",j+1
  end
  H = MPO(os,sites)

  nsweeps = 15 # number of sweeps is 5
  maxdim = [10, 100, 1000, 10000, 25580] # gradually increase states kept
  cutoff = [1E-10] # desired truncation error

  psi0 = random_mps(sites;linkdims=10)

  energy,psi1 = dmrg(H,psi0;nsweeps,maxdim,cutoff)
  
a1=0
orthogonalize!(psi1,A)
ket1 = psi1[A]
for i=A+1:C
    ket1 *= psi1[i]
end
bra1 = dag(ket1)
rho = prime(ket1,"Site")*bra1
@show inds(rho)  #<-- this shows that the indices of rho corresponding to the site between B and C are the 3rd and 7th element of the inds(rho) vector.
@show(delta(inds(rho)[3],inds(rho)[7]))
@show(inds(rho)[3])
@show(inds(rho)[9])
@show(inds(rho)[4])
@show(inds(rho)[10]) 
@show(inds(rho)[5])
@show(inds(rho)[11])     
rho_ABC = rho*delta(inds(rho)[3],inds(rho)[9])     # delta(inds,inds) traces out the given inds (see the documentation)  
rho_ABC1 = rho_ABC*delta(inds(rho)[4],inds(rho)[10])   
rho_ABC2 = rho_ABC*delta(inds(rho)[5],inds(rho)[11])   
da, ua = eigen(rho_ABC2)
for j=1:length(diag(real(da)))
   a1+=diag(real(da))[j]
    end
    @show a1;
    return
end

After sweep 1 energy=-6.9081378101629225  maxlinkdim=10 maxerr=1.62E-03 time=0.047
After sweep 2 energy=-6.91173677865433  maxlinkdim=34 maxerr=8.35E-11 time=0.071
After sweep 3 energy=-6.911737143612211  maxlinkdim=36 maxerr=9.45E-11 time=0.169
After sweep 4 energy=-6.911737143345142  maxlinkdim=36 maxerr=9.93E-11 time=0.148
After sweep 5 energy=-6.911737143346203  maxlinkdim=36 maxerr=9.87E-11 time=0.133
After sweep 6 energy=-6.911737143345912  maxlinkdim=36 maxerr=9.87E-11 time=0.132
After sweep 7 energy=-6.911737143345355  maxlinkdim=36 maxerr=9.87E-11 time=0.134
After sweep 8 energy=-6.9117371433447214  maxlinkdim=36 maxerr=9.87E-11 time=0.131
After sweep 9 energy=-6.911737143344359  maxlinkdim=36 maxerr=9.87E-11 time=0.119
After sweep 10 energy=-6.9117371433442525  maxlinkdim=36 maxerr=9.87E-11 time=0.139
After sweep 11 energy=-6.911737143344272  maxlinkdim=36 maxerr=9.87E-11 time=0.131
After sweep 12 energy=-6.911737143344354  maxlinkdim=36 maxerr=9.87E-11 time=0.120
After sweep

In [6]:
```
K=0
```

`K=0`

In [7]:
let
    A=1
    D=4
  N = 16
  sites = siteinds("S=1/2",N)

  os = OpSum()
  for j=1:N-1
    os += 0.5,"S+",j,"S-",j+1
    os += 0.5,"S-",j,"S+",j+1
    os += "Sz",j,"Sz",j+1
  end
  H = MPO(os,sites)

  nsweeps = 15 # number of sweeps is 5
  maxdim = [10, 100, 1000, 10000, 25580] # gradually increase states kept
  cutoff = [1E-10] # desired truncation error

  psi0 = random_mps(sites;linkdims=10)

  energy,psi1 = dmrg(H,psi0;nsweeps,maxdim,cutoff)
  
a1=0
N1=0
orthogonalize!(psi1,A)
ket1 = psi1[A]
for i=A+1:D
    ket1 *= psi1[i]
end
bra1 = dag(ket1)
rho = prime(ket1,"Site")*bra1
@show inds(rho)  #<-- this shows that the indices of rho corresponding to the site between B and C are the 3rd and 7th element of the inds(rho) vector.    
da, ua = eigen(rho)
    @show diag(da)
for j=1:length(diag(real(da)))
   a1+=diag(real(da))[j]
    end
    @show a1;
    
for j=1:length(diag(real(da)))
   N1+=0.5*(abs(diag(real(da))[j])-diag(real(da))[j])       
    end 
L1= log(1+2*N1)
    @show N1;
  return
end

After sweep 1 energy=-6.909982726480691  maxlinkdim=10 maxerr=3.93E-04 time=0.021
After sweep 2 energy=-6.911737047341471  maxlinkdim=33 maxerr=9.34E-11 time=0.065
After sweep 3 energy=-6.911737143614733  maxlinkdim=36 maxerr=8.95E-11 time=0.232
After sweep 4 energy=-6.91173714334291  maxlinkdim=36 maxerr=9.93E-11 time=0.135
After sweep 5 energy=-6.911737143343085  maxlinkdim=36 maxerr=9.88E-11 time=0.135
After sweep 6 energy=-6.911737143343698  maxlinkdim=36 maxerr=9.88E-11 time=0.119
After sweep 7 energy=-6.911737143344595  maxlinkdim=36 maxerr=9.88E-11 time=0.134
After sweep 8 energy=-6.911737143344698  maxlinkdim=36 maxerr=9.88E-11 time=0.117
After sweep 9 energy=-6.91173714334435  maxlinkdim=36 maxerr=9.88E-11 time=0.138
After sweep 10 energy=-6.911737143343796  maxlinkdim=36 maxerr=9.88E-11 time=0.134
After sweep 11 energy=-6.911737143343537  maxlinkdim=36 maxerr=9.88E-11 time=0.119
After sweep 12 energy=-6.911737143343858  maxlinkdim=36 maxerr=9.88E-11 time=0.136
After sweep 13 

In [8]:
```
K=1
```

`K=1`

In [9]:
let
    A=1
    D=5
  N = 16
  sites = siteinds("S=1/2",N)

  os = OpSum()
  for j=1:N-1
    os += 0.5,"S+",j,"S-",j+1
    os += 0.5,"S-",j,"S+",j+1
    os += "Sz",j,"Sz",j+1
  end
  H = MPO(os,sites)

  nsweeps = 15 # number of sweeps is 5
  maxdim = [10, 100, 1000, 10000, 25580] # gradually increase states kept
  cutoff = [1E-10] # desired truncation error

  psi0 = random_mps(sites;linkdims=10)

  energy,psi1 = dmrg(H,psi0;nsweeps,maxdim,cutoff)
  
a2=0
N2=0
orthogonalize!(psi1,A)
ket1 = psi1[A]
for i=A+1:D
    ket1 *= psi1[i]
end
bra1 = dag(ket1)
rho = prime(ket1,"Site")*bra1
@show inds(rho)  #<-- this shows that the indices of rho corresponding to the site between B and C are the 3rd and 7th element of the inds(rho) vector.    
rho_ABC = rho*delta(inds(rho)[3],inds(rho)[8])     # delta(inds,inds) traces out the given inds (see the documentation)    
da1, ua1 = eigen(rho_ABC)
@show inds(rho_ABC)
for j=1:length(diag(real(da1)))
   a2+=diag(real(da1))[j]
    end
    @show a2;
for j=1:length(diag(real(da1)))
   N2+=0.5*(abs(diag(real(da1))[j])-diag(real(da1))[j])       
    end 
L2= log(1+2*N2)
    @show N2;
  return
end

After sweep 1 energy=-6.9099684625447875  maxlinkdim=10 maxerr=1.39E-03 time=0.022
After sweep 2 energy=-6.91173704359588  maxlinkdim=33 maxerr=9.63E-11 time=0.100
After sweep 3 energy=-6.911737143348782  maxlinkdim=36 maxerr=9.99E-11 time=0.191
After sweep 4 energy=-6.911737143346604  maxlinkdim=36 maxerr=9.88E-11 time=0.151
After sweep 5 energy=-6.911737143346367  maxlinkdim=36 maxerr=9.87E-11 time=0.120
After sweep 6 energy=-6.911737143346697  maxlinkdim=36 maxerr=9.87E-11 time=0.134
After sweep 7 energy=-6.91173714334704  maxlinkdim=36 maxerr=9.87E-11 time=0.127
After sweep 8 energy=-6.911737143347395  maxlinkdim=36 maxerr=9.87E-11 time=0.135
After sweep 9 energy=-6.911737143347734  maxlinkdim=36 maxerr=9.87E-11 time=0.116
After sweep 10 energy=-6.911737143348022  maxlinkdim=36 maxerr=9.87E-11 time=0.137
After sweep 11 energy=-6.911737143348267  maxlinkdim=36 maxerr=9.87E-11 time=0.121
After sweep 12 energy=-6.911737143348482  maxlinkdim=36 maxerr=9.87E-11 time=0.133
After sweep 13

In [10]:
```
K=2
``` 

`K=2`

In [11]:
let
    A=1
    D=6
  N = 16
  sites = siteinds("S=1/2",N)

  os = OpSum()
  for j=1:N-1
    os += 0.5,"S+",j,"S-",j+1
    os += 0.5,"S-",j,"S+",j+1
    os += "Sz",j,"Sz",j+1
  end
  H = MPO(os,sites)

  nsweeps = 15 # number of sweeps is 5
  maxdim = [10, 100, 1000, 10000, 25580] # gradually increase states kept
  cutoff = [1E-10] # desired truncation error

  psi0 = random_mps(sites;linkdims=10)

  energy,psi1 = dmrg(H,psi0;nsweeps,maxdim,cutoff)
  
a1=0
orthogonalize!(psi1,A)
ket1 = psi1[A]
for i=A+1:D
    ket1 *= psi1[i]
end
bra1 = dag(ket1)
rho = prime(ket1,"Site")*bra1
@show inds(rho)  #<-- this shows that the indices of rho corresponding to the site between B and C are the 3rd and 7th element of the inds(rho) vector.    
rho_ABC = rho*delta(inds(rho)[3],inds(rho)[9])     # delta(inds,inds) traces out the given inds (see the documentation)  
rho_ABC1 = rho_ABC*delta(inds(rho)[4],inds(rho)[10])   
#rho_ABC2 = rho_ABC*delta(inds(rho)[5],inds(rho)[11])   
da, ua = eigen(rho_ABC1)
for j=1:length(diag(real(da)))
   a1+=diag(real(da))[j]
    end
    @show a1;
    return
end

After sweep 1 energy=-6.90834595463212  maxlinkdim=10 maxerr=1.29E-03 time=0.048
After sweep 2 energy=-6.911736836068348  maxlinkdim=34 maxerr=9.99E-11 time=0.067
After sweep 3 energy=-6.9117371436097415  maxlinkdim=36 maxerr=9.26E-11 time=0.142
After sweep 4 energy=-6.91173714334705  maxlinkdim=36 maxerr=9.93E-11 time=0.174
After sweep 5 energy=-6.911737143341792  maxlinkdim=36 maxerr=9.87E-11 time=0.117
After sweep 6 energy=-6.911737143339936  maxlinkdim=36 maxerr=9.87E-11 time=0.140
After sweep 7 energy=-6.911737143343814  maxlinkdim=36 maxerr=9.87E-11 time=0.137
After sweep 8 energy=-6.911737143346087  maxlinkdim=36 maxerr=9.87E-11 time=0.116
After sweep 9 energy=-6.911737143344425  maxlinkdim=36 maxerr=9.87E-11 time=0.130
After sweep 10 energy=-6.911737143343116  maxlinkdim=36 maxerr=9.87E-11 time=0.124
After sweep 11 energy=-6.9117371433427195  maxlinkdim=36 maxerr=9.87E-11 time=0.127
After sweep 12 energy=-6.91173714334265  maxlinkdim=36 maxerr=9.87E-11 time=0.122
After sweep 13

In [12]:
using ITensors

# Define constants
A = 1
D = 4
N = 16
sites = siteinds("S=1/2", N)

# Define the Hamiltonian
os = OpSum()
for j = 1:N-1
    os += 0.5, "S+", j, "S-", j+1
    os += 0.5, "S-", j, "S+", j+1
    os += "Sz", j, "Sz", j+1
end
H = MPO(os, sites)

# DMRG parameters
nsweeps = 15
maxdim = [10, 100, 1000, 10000, 25580]
cutoff = [1E-10]

# Random initial state
psi0 = randomMPS(sites; linkdims=10)

# Perform DMRG
energy, psi1 = dmrg(H, psi0; nsweeps, maxdim, cutoff)

# Create the density matrix rho
a1 = 0
N1 = 0
orthogonalize!(psi1, A)
ket1 = psi1[A]
for i = A+1:D
    ket1 *= psi1[i]
end
bra1 = dag(ket1)
rho = prime(ket1, "Site") * bra1
@show inds(rho)
da11, ua11 = eigen(rho)
@show real(diag(da11))

oldinds = inds(rho)
newinds = [oldinds[5],oldinds[6],oldinds[3],oldinds[4],oldinds[1], oldinds[2], oldinds[7], oldinds[8]]

rho_pt = swapinds(rho,oldinds,newinds)
@show inds(rho_pt)
# Compute the eigenvalues of the partially transposed density matrix
da, ua = eigen(rho_pt)
@show real(diag(da))

# Calculate the negativity
for j = 1:length(diag(real(da)))
    a1 += diag(real(da))[j]
end
@show a1

for j = 1:length(diag(real(da)))
    N1 += 0.5 * (abs(diag(real(da))[j]) - diag(real(da))[j])
end
L1 = log(1 + 2 * N1)
@show N1
@show L1

After sweep 1 energy=-6.908470283596092  maxlinkdim=10 maxerr=1.62E-03 time=0.026
After sweep 2 energy=-6.911736841608898  maxlinkdim=33 maxerr=9.66E-11 time=0.073
After sweep 3 energy=-6.911737143614933  maxlinkdim=36 maxerr=8.03E-11 time=0.171
After sweep 4 energy=-6.911737143344437  maxlinkdim=36 maxerr=9.93E-11 time=0.127
After sweep 5 energy=-6.911737143344286  maxlinkdim=36 maxerr=9.88E-11 time=0.140
After sweep 6 energy=-6.911737143344478  maxlinkdim=36 maxerr=9.88E-11 time=0.119
After sweep 7 energy=-6.911737143344661  maxlinkdim=36 maxerr=9.88E-11 time=0.132
After sweep 8 energy=-6.911737143344844  maxlinkdim=36 maxerr=9.88E-11 time=0.119
After sweep 9 energy=-6.911737143344848  maxlinkdim=36 maxerr=9.88E-11 time=0.155
After sweep 10 energy=-6.911737143344483  maxlinkdim=36 maxerr=9.88E-11 time=0.114
After sweep 11 energy=-6.9117371433441965  maxlinkdim=36 maxerr=9.88E-11 time=0.130
After sweep 12 energy=-6.911737143344337  maxlinkdim=36 maxerr=9.88E-11 time=0.115
After sweep 

0.592285440721645

In [13]:
using ITensors

# Define constants
A = 1
D = 4
N = 16
sites = siteinds("S=1/2", N)

# Define the Hamiltonian
os = OpSum()
for j = 1:N-1
    os += 0.5, "S+", j, "S-", j+1
    os += 0.5, "S-", j, "S+", j+1
    os += "Sz", j, "Sz", j+1
end
H = MPO(os, sites)

# DMRG parameters
nsweeps = 15
maxdim = [10, 100, 1000, 10000, 25580]
cutoff = [1E-10]

# Random initial state
psi0 = randomMPS(sites; linkdims=10)

# Perform DMRG
energy, psi1 = dmrg(H, psi0; nsweeps, maxdim, cutoff)

# Create the density matrix rho
a1 = 0
N1 = 0
orthogonalize!(psi1, A)
ket1 = psi1[A]
for i = A+1:D
    ket1 *= psi1[i]
end
bra1 = dag(ket1)
rho = prime(ket1, "Site") * bra1
@show inds(rho)
da11, ua11 = eigen(rho)
@show real(diag(da11))

oldinds = inds(rho)
newinds = [oldinds[5],oldinds[2],oldinds[7],oldinds[4],oldinds[1], oldinds[6], oldinds[3], oldinds[8]]

rho_pt = swapinds(rho,oldinds,newinds)
@show inds(rho_pt)
# Compute the eigenvalues of the partially transposed density matrix
da, ua = eigen(rho_pt)
@show real(diag(da))

# Calculate the negativity
for j = 1:length(diag(real(da)))
    a1 += diag(real(da))[j]
end
@show a1

for j = 1:length(diag(real(da)))
    N1 += 0.5 * (abs(diag(real(da))[j]) - diag(real(da))[j])
end
L1 = log(1 + 2 * N1)
@show N1
@show L1

After sweep 1 energy=-6.900257034841986  maxlinkdim=10 maxerr=7.30E-04 time=0.023
After sweep 2 energy=-6.911733595209164  maxlinkdim=35 maxerr=9.76E-11 time=0.066
After sweep 3 energy=-6.911737143606842  maxlinkdim=36 maxerr=9.22E-11 time=0.176
After sweep 4 energy=-6.911737143351866  maxlinkdim=36 maxerr=9.93E-11 time=0.125
After sweep 5 energy=-6.911737143342982  maxlinkdim=36 maxerr=9.88E-11 time=0.139
After sweep 6 energy=-6.911737143342442  maxlinkdim=36 maxerr=9.87E-11 time=0.120
After sweep 7 energy=-6.911737143343174  maxlinkdim=36 maxerr=9.87E-11 time=0.141
After sweep 8 energy=-6.911737143344849  maxlinkdim=36 maxerr=9.87E-11 time=0.115
After sweep 9 energy=-6.911737143345767  maxlinkdim=36 maxerr=9.87E-11 time=0.130
After sweep 10 energy=-6.911737143345611  maxlinkdim=36 maxerr=9.87E-11 time=0.124
After sweep 11 energy=-6.911737143345139  maxlinkdim=36 maxerr=9.87E-11 time=0.131
After sweep 12 energy=-6.911737143344746  maxlinkdim=36 maxerr=9.87E-11 time=0.117
After sweep 1

1.168920017328076

In [14]:
using ITensors

# Define constants
A = 1
D = 4
N = 16
sites = siteinds("S=1/2", N)

# Define the Hamiltonian
os = OpSum()
for j = 1:N-1
    os += 0.5, "S+", j, "S-", j+1
    os += 0.5, "S-", j, "S+", j+1
    os += "Sz", j, "Sz", j+1
end
H = MPO(os, sites)

# DMRG parameters
nsweeps = 15
maxdim = [10, 100, 1000, 10000, 25580]
cutoff = [1E-10]

# Random initial state
psi0 = randomMPS(sites; linkdims=10)

# Perform DMRG
energy, psi1 = dmrg(H, psi0; nsweeps, maxdim, cutoff)

# Create the density matrix rho
a1 = 0
N1 = 0
orthogonalize!(psi1, A)
ket1 = psi1[A]
for i = A+1:D
    ket1 *= psi1[i]
end
bra1 = dag(ket1)
rho = prime(ket1, "Site") * bra1
@show inds(rho)
da11, ua11 = eigen(rho)
@show real(diag(da11))

oldinds = inds(rho)
newinds = [oldinds[5],oldinds[2],oldinds[3],oldinds[8],oldinds[1], oldinds[6], oldinds[7], oldinds[4]]

rho_pt = swapinds(rho,oldinds,newinds)
@show inds(rho_pt)
# Compute the eigenvalues of the partially transposed density matrix
da, ua = eigen(rho_pt)
@show real(diag(da))

# Calculate the negativity
for j = 1:length(diag(real(da)))
    a1 += diag(real(da))[j]
end
@show a1

for j = 1:length(diag(real(da)))
    N1 += 0.5 * (abs(diag(real(da))[j]) - diag(real(da))[j])
end
L1 = log(1 + 2 * N1)
@show N1
@show L1

After sweep 1 energy=-6.909056973550568  maxlinkdim=10 maxerr=3.87E-03 time=0.033
After sweep 2 energy=-6.911736846550971  maxlinkdim=34 maxerr=9.85E-11 time=0.080
After sweep 3 energy=-6.911737143612715  maxlinkdim=36 maxerr=9.18E-11 time=0.149
After sweep 4 energy=-6.9117371433443555  maxlinkdim=36 maxerr=9.93E-11 time=0.162
After sweep 5 energy=-6.911737143348088  maxlinkdim=36 maxerr=9.87E-11 time=0.137
After sweep 6 energy=-6.911737143350516  maxlinkdim=36 maxerr=9.87E-11 time=0.129
After sweep 7 energy=-6.911737143351571  maxlinkdim=36 maxerr=9.87E-11 time=0.126
After sweep 8 energy=-6.911737143351926  maxlinkdim=36 maxerr=9.87E-11 time=0.123
After sweep 9 energy=-6.911737143352591  maxlinkdim=36 maxerr=9.87E-11 time=0.127
After sweep 10 energy=-6.911737143353188  maxlinkdim=36 maxerr=9.87E-11 time=0.129
After sweep 11 energy=-6.911737143352189  maxlinkdim=36 maxerr=9.87E-11 time=0.126
After sweep 12 energy=-6.911737143349718  maxlinkdim=36 maxerr=9.88E-11 time=0.116
After sweep 

1.1686137963548509

In [15]:
using ITensors

# Define constants
A = 1
D = 4
N = 16
sites = siteinds("S=1/2", N)

# Define the Hamiltonian
os = OpSum()
for j = 1:N-1
    os += 0.5, "S+", j, "S-", j+1
    os += 0.5, "S-", j, "S+", j+1
    os += "Sz", j, "Sz", j+1
end
H = MPO(os, sites)

# DMRG parameters
nsweeps = 15
maxdim = [10, 100, 1000, 10000, 25580]
cutoff = [1E-10]

# Random initial state
psi0 = randomMPS(sites; linkdims=10)

# Perform DMRG
energy, psi1 = dmrg(H, psi0; nsweeps, maxdim, cutoff)

# Create the density matrix rho
a1 = 0
N1 = 0
orthogonalize!(psi1, A)
ket1 = psi1[A]
for i = A+1:D
    ket1 *= psi1[i]
end
bra1 = dag(ket1)
rho = prime(ket1, "Site") * bra1
@show inds(rho)
da11, ua11 = eigen(rho)
@show real(diag(da11))

oldinds = inds(rho)
newinds = [oldinds[5],oldinds[2],oldinds[3],oldinds[4],oldinds[1], oldinds[6], oldinds[7], oldinds[8]]

rho_pt = swapinds(rho,oldinds,newinds)
@show inds(rho_pt)
# Compute the eigenvalues of the partially transposed density matrix
da, ua = eigen(rho_pt)
@show real(diag(da))

# Calculate the negativity
for j = 1:length(diag(real(da)))
    a1 += diag(real(da))[j]
end
@show a1

for j = 1:length(diag(real(da)))
    N1 += 0.5 * (abs(diag(real(da))[j]) - diag(real(da))[j])
end
L1 = log(1 + 2 * N1)
@show N1
@show L1

After sweep 1 energy=-6.908274840022051  maxlinkdim=10 maxerr=8.83E-04 time=0.039
After sweep 2 energy=-6.911736919644086  maxlinkdim=32 maxerr=9.48E-11 time=0.076
After sweep 3 energy=-6.91173714360608  maxlinkdim=36 maxerr=9.92E-11 time=0.142
After sweep 4 energy=-6.911737143340918  maxlinkdim=36 maxerr=9.93E-11 time=0.151
After sweep 5 energy=-6.91173714334188  maxlinkdim=36 maxerr=9.88E-11 time=0.136
After sweep 6 energy=-6.911737143342561  maxlinkdim=36 maxerr=9.88E-11 time=0.138
After sweep 7 energy=-6.911737143341846  maxlinkdim=36 maxerr=9.88E-11 time=0.113
After sweep 8 energy=-6.911737143343495  maxlinkdim=36 maxerr=9.88E-11 time=0.133
After sweep 9 energy=-6.911737143345962  maxlinkdim=36 maxerr=9.88E-11 time=0.133
After sweep 10 energy=-6.9117371433451265  maxlinkdim=36 maxerr=9.88E-11 time=0.132
After sweep 11 energy=-6.9117371433458406  maxlinkdim=36 maxerr=9.88E-11 time=0.111
After sweep 12 energy=-6.911737143345787  maxlinkdim=36 maxerr=9.88E-11 time=0.127
After sweep 1

0.6486868747591611

In [16]:
using ITensors

# Define constants
A = 1
D = 4
N = 16
sites = siteinds("S=1/2", N)

# Define the Hamiltonian
os = OpSum()
for j = 1:N-1
    os += 0.5, "S+", j, "S-", j+1
    os += 0.5, "S-", j, "S+", j+1
    os += "Sz", j, "Sz", j+1
end
H = MPO(os, sites)

# DMRG parameters
nsweeps = 15
maxdim = [10, 100, 1000, 10000, 25580]
cutoff = [1E-10]

# Random initial state
psi0 = randomMPS(sites; linkdims=10)

# Perform DMRG
energy, psi1 = dmrg(H, psi0; nsweeps, maxdim, cutoff)

# Create the density matrix rho
a1 = 0
N1 = 0
orthogonalize!(psi1, A)
ket1 = psi1[A]
for i = A+1:D
    ket1 *= psi1[i]
end
bra1 = dag(ket1)
rho = prime(ket1, "Site") * bra1
@show inds(rho)
da11, ua11 = eigen(rho)
@show real(diag(da11))

oldinds = inds(rho)
newinds = [oldinds[1],oldinds[6],oldinds[3],oldinds[4],oldinds[5], oldinds[2], oldinds[7], oldinds[8]]

rho_pt = swapinds(rho,oldinds,newinds)
@show inds(rho_pt)
# Compute the eigenvalues of the partially transposed density matrix
da, ua = eigen(rho_pt)
@show real(diag(da))

# Calculate the negativity
for j = 1:length(diag(real(da)))
    a1 += diag(real(da))[j]
end
@show a1

for j = 1:length(diag(real(da)))
    N1 += 0.5 * (abs(diag(real(da))[j]) - diag(real(da))[j])
end
L1 = log(1 + 2 * N1)
@show N1
@show L1

After sweep 1 energy=-6.905803822636611  maxlinkdim=10 maxerr=4.62E-04 time=0.036
After sweep 2 energy=-6.911736321562009  maxlinkdim=33 maxerr=9.87E-11 time=0.084
After sweep 3 energy=-6.91173714360622  maxlinkdim=36 maxerr=8.06E-11 time=0.120
After sweep 4 energy=-6.911737143343636  maxlinkdim=36 maxerr=9.93E-11 time=0.165
After sweep 5 energy=-6.911737143345395  maxlinkdim=36 maxerr=9.87E-11 time=0.131
After sweep 6 energy=-6.911737143347391  maxlinkdim=36 maxerr=9.87E-11 time=0.135
After sweep 7 energy=-6.911737143348159  maxlinkdim=36 maxerr=9.87E-11 time=0.113
After sweep 8 energy=-6.911737143348651  maxlinkdim=36 maxerr=9.87E-11 time=0.143
After sweep 9 energy=-6.911737143349171  maxlinkdim=36 maxerr=9.87E-11 time=0.122
After sweep 10 energy=-6.911737143349795  maxlinkdim=36 maxerr=9.87E-11 time=0.130
After sweep 11 energy=-6.911737143350374  maxlinkdim=36 maxerr=9.87E-11 time=0.114
After sweep 12 energy=-6.911737143350907  maxlinkdim=36 maxerr=9.88E-11 time=0.132
After sweep 13

0.6527690518736494

In [17]:
using ITensors

# Define constants
A = 1
D = 4
N = 16
sites = siteinds("S=1/2", N)

# Define the Hamiltonian
os = OpSum()
for j = 1:N-1
    os += 0.5, "S+", j, "S-", j+1
    os += 0.5, "S-", j, "S+", j+1
    os += "Sz", j, "Sz", j+1
end
H = MPO(os, sites)

# DMRG parameters
nsweeps = 15
maxdim = [10, 100, 1000, 10000, 25580]
cutoff = [1E-10]

# Random initial state
psi0 = randomMPS(sites; linkdims=10)

# Perform DMRG
energy, psi1 = dmrg(H, psi0; nsweeps, maxdim, cutoff)

# Create the density matrix rho
a1 = 0
N1 = 0
orthogonalize!(psi1, A)
ket1 = psi1[A]
for i = A+1:D
    ket1 *= psi1[i]
end
bra1 = dag(ket1)
rho = prime(ket1, "Site") * bra1
@show inds(rho)
da11, ua11 = eigen(rho)
@show real(diag(da11))

oldinds = inds(rho)
newinds = [oldinds[1],oldinds[2],oldinds[7],oldinds[4],oldinds[5], oldinds[6], oldinds[3], oldinds[8]]

rho_pt = swapinds(rho,oldinds,newinds)
@show inds(rho_pt)
# Compute the eigenvalues of the partially transposed density matrix
da, ua = eigen(rho_pt)
@show real(diag(da))

# Calculate the negativity
for j = 1:length(diag(real(da)))
    a1 += diag(real(da))[j]
end
@show a1

for j = 1:length(diag(real(da)))
    N1 += 0.5 * (abs(diag(real(da))[j]) - diag(real(da))[j])
end
L1 = log(1 + 2 * N1)
@show N1
@show L1

After sweep 1 energy=-6.909016888145956  maxlinkdim=10 maxerr=7.63E-04 time=0.023
After sweep 2 energy=-6.91173698227902  maxlinkdim=34 maxerr=9.10E-11 time=0.076
After sweep 3 energy=-6.911737143607016  maxlinkdim=36 maxerr=9.07E-11 time=0.131
After sweep 4 energy=-6.911737143339055  maxlinkdim=36 maxerr=9.93E-11 time=0.118
After sweep 5 energy=-6.91173714333971  maxlinkdim=36 maxerr=9.87E-11 time=0.151
After sweep 6 energy=-6.911737143338822  maxlinkdim=36 maxerr=9.87E-11 time=0.121
After sweep 7 energy=-6.911737143338954  maxlinkdim=36 maxerr=9.88E-11 time=0.131
After sweep 8 energy=-6.911737143339152  maxlinkdim=36 maxerr=9.88E-11 time=0.112
After sweep 9 energy=-6.9117371433393435  maxlinkdim=36 maxerr=9.88E-11 time=0.125
After sweep 10 energy=-6.9117371433395105  maxlinkdim=36 maxerr=9.88E-11 time=0.115
After sweep 11 energy=-6.911737143339737  maxlinkdim=36 maxerr=9.88E-11 time=0.139
After sweep 12 energy=-6.911737143339844  maxlinkdim=36 maxerr=9.88E-11 time=0.109
After sweep 1

0.6021286718421678

In [18]:
using ITensors

# Define constants
A = 1
D = 4
N = 16
sites = siteinds("S=1/2", N)

# Define the Hamiltonian
os = OpSum()
for j = 1:N-1
    os += 0.5, "S+", j, "S-", j+1
    os += 0.5, "S-", j, "S+", j+1
    os += "Sz", j, "Sz", j+1
end
H = MPO(os, sites)

# DMRG parameters
nsweeps = 15
maxdim = [10, 100, 1000, 10000, 25580]
cutoff = [1E-10]

# Random initial state
psi0 = randomMPS(sites; linkdims=10)

# Perform DMRG
energy, psi1 = dmrg(H, psi0; nsweeps, maxdim, cutoff)

# Create the density matrix rho
a1 = 0
N1 = 0
orthogonalize!(psi1, A)
ket1 = psi1[A]
for i = A+1:D
    ket1 *= psi1[i]
end
bra1 = dag(ket1)
rho = prime(ket1, "Site") * bra1
@show inds(rho)
da11, ua11 = eigen(rho)
@show real(diag(da11))

oldinds = inds(rho)
newinds = [oldinds[1],oldinds[2],oldinds[3],oldinds[8],oldinds[5], oldinds[6], oldinds[7], oldinds[4]]

rho_pt = swapinds(rho,oldinds,newinds)
@show inds(rho_pt)
# Compute the eigenvalues of the partially transposed density matrix
da, ua = eigen(rho_pt)
@show real(diag(da))

# Calculate the negativity
for j = 1:length(diag(real(da)))
    a1 += diag(real(da))[j]
end
@show a1

for j = 1:length(diag(real(da)))
    N1 += 0.5 * (abs(diag(real(da))[j]) - diag(real(da))[j])
end
L1 = log(1 + 2 * N1)
@show N1
@show L1

After sweep 1 energy=-6.909802885995147  maxlinkdim=10 maxerr=1.45E-03 time=0.040
After sweep 2 energy=-6.9117370407873935  maxlinkdim=32 maxerr=9.94E-11 time=0.083
After sweep 3 energy=-6.911737143610523  maxlinkdim=36 maxerr=8.83E-11 time=0.145
After sweep 4 energy=-6.91173714334086  maxlinkdim=36 maxerr=9.93E-11 time=0.163
After sweep 5 energy=-6.911737143340954  maxlinkdim=36 maxerr=9.87E-11 time=0.139
After sweep 6 energy=-6.911737143340833  maxlinkdim=36 maxerr=9.87E-11 time=0.116
After sweep 7 energy=-6.911737143340582  maxlinkdim=36 maxerr=9.87E-11 time=0.140
After sweep 8 energy=-6.911737143340272  maxlinkdim=36 maxerr=9.87E-11 time=0.108
After sweep 9 energy=-6.9117371433399555  maxlinkdim=36 maxerr=9.87E-11 time=0.133
After sweep 10 energy=-6.911737143339549  maxlinkdim=36 maxerr=9.87E-11 time=0.275
After sweep 11 energy=-6.91173714333903  maxlinkdim=36 maxerr=9.87E-11 time=0.156
After sweep 12 energy=-6.911737143338446  maxlinkdim=36 maxerr=9.87E-11 time=0.176
After sweep 1

0.5599663594685436

In [19]:
(0.5922854408696039*1.168920017300053*1.1686137963985457*0.6486868747406122*0.6527690518829391*0.6021286718076799*0.5599663594221891)^(1/7)

0.7346667695198645

In [20]:
using ITensors

# Define constants
A = 1
D = 4
N = 16
sites = siteinds("S=1/2", N)

# Define the Hamiltonian
os = OpSum()
for j = 1:N-1
    os += 0.5, "S+", j, "S-", j+1
    os += 0.5, "S-", j, "S+", j+1
    os += "Sz", j, "Sz", j+1
end

os += 0.5, "S+", N, "S-", 1
os += 0.5, "S-", N, "S+", 1
os += "Sz", N, "Sz", 1

H = MPO(os, sites)

# DMRG parameters
nsweeps = 10
maxdim = [10, 100, 1000, 10000, 25580]
cutoff = [1E-10]

# Random initial state
psi0 = randomMPS(sites; linkdims=10)

# Perform DMRG
energy, psi1 = dmrg(H, psi0; nsweeps, maxdim, cutoff)

# Create the density matrix rho
a1 = 0
N1 = 0
a2 = 0
N2 = 0
a3 = 0
N3 = 0
a4 = 0
N4 = 0
a5 = 0
N5 = 0
a6 = 0
N6 = 0
a7 = 0
N7 = 0
orthogonalize!(psi1, A)
ket1 = psi1[A]
for i = A+1:D
    ket1 *= psi1[i]
end
bra1 = dag(ket1)
rho = prime(ket1, "Site") * bra1

da11, ua11 = eigen(rho)


oldinds = inds(rho)
newinds = [oldinds[5],oldinds[6],oldinds[3],oldinds[4],oldinds[1], oldinds[2], oldinds[7], oldinds[8]]
newinds1 = [oldinds[5],oldinds[2],oldinds[7],oldinds[4],oldinds[1], oldinds[6], oldinds[3], oldinds[8]]
newinds2 = [oldinds[5],oldinds[2],oldinds[3],oldinds[8],oldinds[1], oldinds[6], oldinds[7], oldinds[4]]
newinds3 = [oldinds[5],oldinds[2],oldinds[3],oldinds[4],oldinds[1], oldinds[6], oldinds[7], oldinds[8]]
newinds4 = [oldinds[1],oldinds[6],oldinds[3],oldinds[4],oldinds[5], oldinds[2], oldinds[7], oldinds[8]]
newinds5 = [oldinds[1],oldinds[2],oldinds[7],oldinds[4],oldinds[5], oldinds[6], oldinds[3], oldinds[8]]
newinds6 = [oldinds[1],oldinds[2],oldinds[3],oldinds[8],oldinds[5], oldinds[6], oldinds[7], oldinds[4]]





rho_pt = swapinds(rho,oldinds,newinds)
rho_pt1 = swapinds(rho,oldinds,newinds1)
rho_pt2 = swapinds(rho,oldinds,newinds2)
rho_pt3 = swapinds(rho,oldinds,newinds3)
rho_pt4 = swapinds(rho,oldinds,newinds4)
rho_pt5 = swapinds(rho,oldinds,newinds5)
rho_pt6 = swapinds(rho,oldinds,newinds6)

# Compute the eigenvalues of the partially transposed density matrix
da, ua = eigen(rho_pt)
da1, ua1 = eigen(rho_pt1)
da2, ua2 = eigen(rho_pt2)
da3, ua3 = eigen(rho_pt3)
da4, u4 = eigen(rho_pt4)
da5, ua5 = eigen(rho_pt5)
da6, ua6 = eigen(rho_pt6)

# Calculate the negativity
for j = 1:length(diag(real(da)))
    a1 += diag(real(da))[j]
end


for j = 1:length(diag(real(da)))
    N1 += 0.5 * (abs(diag(real(da))[j]) - diag(real(da))[j])
end
L1 = log(1 + 2 * N1)


for j = 1:length(diag(real(da1)))
    N2 += 0.5 * (abs(diag(real(da1))[j]) - diag(real(da1))[j])
end
L2 = log(1 + 2 * N2)

for j = 1:length(diag(real(da2)))
    N3 += 0.5 * (abs(diag(real(da2))[j]) - diag(real(da2))[j])
end
L3 = log(1 + 2 * N3)


for j = 1:length(diag(real(da3)))
    N4 += 0.5 * (abs(diag(real(da3))[j]) - diag(real(da3))[j])
end
L4 = log(1 + 2 * N4)


for j = 1:length(diag(real(da4)))
    N5 += 0.5 * (abs(diag(real(da4))[j]) - diag(real(da4))[j])
end
L5 = log(1 + 2 * N5)

for j = 1:length(diag(real(da5)))
    N6 += 0.5 * (abs(diag(real(da5))[j]) - diag(real(da5))[j])
end
L6 = log(1 + 2 * N6)


for j = 1:length(diag(real(da6)))
    N7 += 0.5 * (abs(diag(real(da6))[j]) - diag(real(da6))[j])
end
L7 = log(1 + 2 * N7)

L11=(L1*L2*L3*L4*L5*L6*L7)^(1/7)
@show L11;

After sweep 1 energy=-7.079699472357534  maxlinkdim=10 maxerr=1.17E-03 time=0.043
After sweep 2 energy=-7.142223122529169  maxlinkdim=40 maxerr=4.63E-16 time=0.276
After sweep 3 energy=-7.14229635798646  maxlinkdim=116 maxerr=9.97E-11 time=1.158
After sweep 4 energy=-7.142296358603551  maxlinkdim=121 maxerr=9.60E-11 time=1.350
After sweep 5 energy=-7.142296358600866  maxlinkdim=121 maxerr=9.59E-11 time=2.182
After sweep 6 energy=-7.14229635859971  maxlinkdim=121 maxerr=9.59E-11 time=0.527
After sweep 7 energy=-7.142296358599581  maxlinkdim=121 maxerr=9.59E-11 time=0.550
After sweep 8 energy=-7.142296358599571  maxlinkdim=121 maxerr=9.59E-11 time=0.478
After sweep 9 energy=-7.142296358599559  maxlinkdim=121 maxerr=9.59E-11 time=0.483
After sweep 10 energy=-7.142296358599574  maxlinkdim=121 maxerr=9.59E-11 time=0.490
L11 = 0.5391934316524187


In [21]:
0.5366296725831321

0.5366296725831321

In [22]:
using ITensors
ITensors.disable_warn_order()
# Define constants
A = 1
D = 8
N = 16
sites = siteinds("S=1/2", N)

# Define the Hamiltonian
os = OpSum()
for j = 1:N-1
    os += 0.5, "S+", j, "S-", j+1
    os += 0.5, "S-", j, "S+", j+1
    os += "Sz", j, "Sz", j+1
end

os += 0.5, "S+", N, "S-", 1
os += 0.5, "S-", N, "S+", 1
os += "Sz", N, "Sz", 1

H = MPO(os, sites)

# DMRG parameters
nsweeps = 10
maxdim = [10, 100, 1000, 10000, 25580]
cutoff = [1E-10]

# Random initial state
psi0 = randomMPS(sites; linkdims=10)

# Perform DMRG
energy, psi1 = dmrg(H, psi0; nsweeps, maxdim, cutoff)

# Create the density matrix rho
a1 = 0
N1 = 0
a2 = 0
N2 = 0
a3 = 0
N3 = 0
a4 = 0
N4 = 0
a5 = 0
N5 = 0
a6 = 0
N6 = 0
a7 = 0
N7 = 0
orthogonalize!(psi1,A)
ket1 = psi1[A]
for i=A+1:D
    ket1 *= psi1[i]
end
bra1 = dag(ket1)
rho = prime(ket1,"Site")*bra1
@show inds(rho)  #<-- this shows that the indices of rho corresponding to the site between B and C are the 3rd and 7th element of the inds(rho) vector.    
rho_ABC = rho*delta(inds(rho)[3],inds(rho)[11])     # delta(inds,inds) traces out the given inds (see the documentation)  
rho_ABC1 = rho_ABC*delta(inds(rho)[4],inds(rho)[12])   
rho_ABC2 = rho_ABC1*delta(inds(rho)[5],inds(rho)[13])  
rho_ABC3 = rho_ABC2*delta(inds(rho)[6],inds(rho)[14]) 
#da, ua = eigen(rho_ABC1)


oldinds = inds(rho_ABC3)
newinds = [oldinds[5],oldinds[6],oldinds[3],oldinds[4],oldinds[1], oldinds[2], oldinds[7], oldinds[8]]
newinds1 = [oldinds[5],oldinds[2],oldinds[7],oldinds[4],oldinds[1], oldinds[6], oldinds[3], oldinds[8]]
newinds2 = [oldinds[5],oldinds[2],oldinds[3],oldinds[8],oldinds[1], oldinds[6], oldinds[7], oldinds[4]]
newinds3 = [oldinds[5],oldinds[2],oldinds[3],oldinds[4],oldinds[1], oldinds[6], oldinds[7], oldinds[8]]
newinds4 = [oldinds[1],oldinds[6],oldinds[3],oldinds[4],oldinds[5], oldinds[2], oldinds[7], oldinds[8]]
newinds5 = [oldinds[1],oldinds[2],oldinds[7],oldinds[4],oldinds[5], oldinds[6], oldinds[3], oldinds[8]]
newinds6 = [oldinds[1],oldinds[2],oldinds[3],oldinds[8],oldinds[5], oldinds[6], oldinds[7], oldinds[4]]





rho_pt = swapinds(rho_ABC3,oldinds,newinds)
rho_pt1 = swapinds(rho_ABC3,oldinds,newinds1)
rho_pt2 = swapinds(rho_ABC3,oldinds,newinds2)
rho_pt3 = swapinds(rho_ABC3,oldinds,newinds3)
rho_pt4 = swapinds(rho_ABC3,oldinds,newinds4)
rho_pt5 = swapinds(rho_ABC3,oldinds,newinds5)
rho_pt6 = swapinds(rho_ABC3,oldinds,newinds6)

# Compute the eigenvalues of the partially transposed density matrix
da, ua = eigen(rho_pt)
da1, ua1 = eigen(rho_pt1)
da2, ua2 = eigen(rho_pt2)
da3, ua3 = eigen(rho_pt3)
da4, u4 = eigen(rho_pt4)
da5, ua5 = eigen(rho_pt5)
da6, ua6 = eigen(rho_pt6)

# Calculate the negativity
for j = 1:length(diag(real(da)))
    a1 += diag(real(da))[j]
end


for j = 1:length(diag(real(da)))
    N1 += 0.5 * (abs(diag(real(da))[j]) - diag(real(da))[j])
end
L1 = log(1 + 2 * N1)


for j = 1:length(diag(real(da1)))
    N2 += 0.5 * (abs(diag(real(da1))[j]) - diag(real(da1))[j])
end
L2 = log(1 + 2 * N2)

for j = 1:length(diag(real(da2)))
    N3 += 0.5 * (abs(diag(real(da2))[j]) - diag(real(da2))[j])
end
L3 = log(1 + 2 * N3)


for j = 1:length(diag(real(da3)))
    N4 += 0.5 * (abs(diag(real(da3))[j]) - diag(real(da3))[j])
end
L4 = log(1 + 2 * N4)


for j = 1:length(diag(real(da4)))
    N5 += 0.5 * (abs(diag(real(da4))[j]) - diag(real(da4))[j])
end
L5 = log(1 + 2 * N5)

for j = 1:length(diag(real(da5)))
    N6 += 0.5 * (abs(diag(real(da5))[j]) - diag(real(da5))[j])
end
L6 = log(1 + 2 * N6)


for j = 1:length(diag(real(da6)))
    N7 += 0.5 * (abs(diag(real(da6))[j]) - diag(real(da6))[j])
end
L7 = log(1 + 2 * N7)

L11=(L1*L2*L3*L4*L5*L6*L7)^(1/7)
@show L11;

After sweep 1 energy=-7.049984515440114  maxlinkdim=10 maxerr=1.30E-03 time=0.026
After sweep 2 energy=-7.142201170395887  maxlinkdim=40 maxerr=5.53E-16 time=0.125
After sweep 3 energy=-7.142296357752517  maxlinkdim=116 maxerr=9.59E-11 time=0.416
After sweep 4 energy=-7.142296358602739  maxlinkdim=121 maxerr=9.71E-11 time=0.471
After sweep 5 energy=-7.142296358598902  maxlinkdim=121 maxerr=9.59E-11 time=0.477
After sweep 6 energy=-7.142296358598671  maxlinkdim=121 maxerr=9.59E-11 time=0.482
After sweep 7 energy=-7.142296358598633  maxlinkdim=121 maxerr=9.59E-11 time=0.488
After sweep 8 energy=-7.142296358598622  maxlinkdim=121 maxerr=9.59E-11 time=0.484
After sweep 9 energy=-7.142296358598619  maxlinkdim=121 maxerr=9.59E-11 time=0.500
After sweep 10 energy=-7.142296358598616  maxlinkdim=121 maxerr=9.59E-11 time=0.482
inds(rho) = ((dim=2|id=572|"S=1/2,Site,n=1")', (dim=2|id=608|"S=1/2,Site,n=2")', (dim=2|id=9|"S=1/2,Site,n=3")', (dim=2|id=725|"S=1/2,Site,n=4")', (dim=2|id=237|"S=1/2,Sit